<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/3.9.%20%E5%A4%9A%E5%B1%82%E6%84%9F%E7%9F%A5%E6%9C%BA%E7%9A%84%E4%BB%8E%E9%9B%B6%E5%BC%80%E5%A7%8B%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##3.9. 多层感知机的从零开始实现
我们已经从上一节里了解了多层感知机的原理。下面，我们一起来动手实现一个多层感知机。首先导入实现所需的包或模块。

In [0]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython import  display
from tensorflow import keras
import tensorflow.data as tfdata
import numpy as np

In [0]:
tf.enable_eager_execution()

###3.9.1. 获取和读取数据
这里继续使用Fashion-MNIST数据集。我们将使用多层感知机对图像进行分类。

In [0]:
batch_size=256
buffer_size=10000
def load_data_fashion_mnist(batch_size,buffer_size):
  (x_train,y_train),(x_test,y_test)=keras.datasets.fashion_mnist.load_data()
  x_train=np.reshape(x_train,[x_train.shape[0],-1])#将三维张量reshape成二维
  x_test=np.reshape(x_test,[x_test.shape[0],-1])
  train_iter=tfdata.Dataset.from_tensor_slices((x_train,y_train)).map(lambda x,y:(x/255,y)).shuffle(buffer_size).batch(batch_size)
  test_iter=tfdata.Dataset.from_tensor_slices((x_test,y_test)).map(lambda x,y:(x/255,y)).batch(batch_size)
  return train_iter,test_iter

In [0]:
train_iter,test_iter=load_data_fashion_mnist(batch_size=batch_size,buffer_size=buffer_size)

4423680/4422102 [==============================] - 0s 0us/step


###3.9.2. 定义模型参数
我们在“softmax回归的从零开始实现”一节里已经介绍了，Fashion-MNIST数据集中图像形状为28×28，类别数为10。本节中我们依然使用长度为28×28=784的向量表示每一张图像。因此，输入个数为784，输出个数为10。实验中，我们设超参数隐藏单元个数为256。

In [0]:
num_inputs,num_outputs,num_hiddens=784,10,256
W1=tf.Variable(tf.random.normal(shape=(num_inputs,num_hiddens),stddev=0.01))
b1=tf.Variable(tf.zeros(num_hiddens))
W2=tf.Variable(tf.random.normal(shape=(num_hiddens,num_outputs),stddev=0.01))
b2=tf.Variable(tf.zeros(num_outputs))
params=[W1,b1,W2,b2]

###3.9.3. 定义激活函数
这里我们使用基础的maximum函数来实现ReLU，而非直接调用relu函数。

In [0]:
def relu(X):
  return tf.maximum(X,0)

###3.9.4. 定义模型
同softmax回归一样，我们通过reshape函数将每张原始图像改成长度为num_inputs的向量。然后我们实现上一节中多层感知机的计算表达式。

In [0]:
def net(X):
  H=relu(tf.matmul(X,W1)+b1)
  return tf.matmul(H,W2)+b2

###3.9.5. 定义损失函数
为了得到更好的数值稳定性，我们直接使用keras.losses模块提供的包括softmax运算和交叉熵损失计算的函数。

In [0]:
loss=tf.keras.losses.sparse_categorical_crossentropy

###3.9.6. 训练模型
训练多层感知机的步骤和“softmax回归的从零开始实现”一节中训练softmax回归的步骤没什么区别。我们直接使用之前定义的train_ch3函数，它的实现已经在“softmax回归的从零开始实现”一节里介绍过。我们在这里设超参数迭代周期数为5，学习率为0.5。

In [0]:
def sgd(params,loss,t,lr,batch_size):
  for param in params:
    dl_dp=t.gradient(loss,param) #求梯度
    param.assign_sub(lr*dl_dp/batch_size) #更新梯度

def evaluate_accuracy(data_iter,net):
  acc_sum,n=0.0,0
  for X,y in data_iter:
    # y=tf.cast(y,tf.float32)
    acc_sum+=tf.equal(tf.cast(tf.argmax(net(X),axis=1),tf.float32),tf.cast(y,tf.float32)).numpy().sum()
    n+=tf.shape(y)[0].numpy()
  return acc_sum/n

def train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,trainer=None):
  for epoch in range(num_epochs):
    train_l_sum,train_acc_sum,n=0.0,0.0,0
    for X,y in train_iter:
      if trainer is None:
        with tf.GradientTape(persistent=True) as t:
          y_hat=net(X)
          l=tf.reduce_sum(loss(y,y_hat,from_logits=True))
        sgd(params,l,t,lr,batch_size)
      else:
        y_hat=net(X)
        l=tf.reduce_sum(loss(y_hat,y))
        trainer.minimize(lambda:loss(net(X),y),global_step=tf.train.get_or_create_global_step())
      train_l_sum+=l.numpy()
      train_acc_sum+=tf.equal(tf.cast(tf.argmax(y_hat,axis=1),tf.float32),tf.cast(y,tf.float32)).numpy().sum()
      n+=tf.shape(y)[0]
    test_acc=evaluate_accuracy(test_iter,net)
    print('epoch %d,loss %.4f,train acc %.3f,test acc %.3f'%(epoch+1,train_l_sum/n,train_acc_sum/n,test_acc))  

In [0]:
num_epochs,lr=5,0.5
train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params,lr)

epoch 1,loss 0.4435,train acc 0.835,test acc 0.841
epoch 2,loss 0.4010,train acc 0.852,test acc 0.855
epoch 3,loss 0.3731,train acc 0.862,test acc 0.864
epoch 4,loss 0.3529,train acc 0.871,test acc 0.868
epoch 5,loss 0.3365,train acc 0.876,test acc 0.869


###3.9.7. 小结
可以通过手动定义模型及其参数来实现简单的多层感知机。
当多层感知机的层数较多时，本节的实现方法会显得较烦琐，例如在定义模型参数的时候。